In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
import os
!git clone https://github.com/ibrahim-elsawy/dstilPegasus
# !pip install -r /content/konafa/colab_req.txt
!pip install datasets
#!pip install torch==1.7.1
!pip install transformers==4.3.0rc1
!pip install sentencepiece 
!pip install catalyst==20.6
!pip install rouge-score
!pip install sacrebleu

fatal: destination path 'dstilPegasus' already exists and is not an empty directory.


In [ ]:
os.chdir('/content/dstilPegasus')
import torch
import transformers
from catalyst import dl
from src.runners import DistilMLMRunner
from src.runners import DistilMLMRunnerFT
# from src.models import DistilpegasusStudentModel, PegasusForMLM #BertForMLM  DistilbertStudentModel
from catalyst.core import MetricAggregationCallback
from torch.utils.data import DataLoader
from src.callbacks import (
    CosineLossCallback,
    KLDivLossCallback,
    MaskedLanguageModelCallback,
    MSELossCallback,
    PerplexityMetricCallbackDistillation,
    SmoothingLossCallback,
    CrossentropylossCallback
)
from utils.rouge import calc_generative_metrics 
import pandas as pd
from typing import Iterable, Union

# import torch
from torch.utils.data import Dataset
from tqdm.auto import tqdm
# import transformers
from transformers import AutoTokenizer
from transformers.data.data_collator import DataCollatorForLanguageModeling
import numpy as np
from transformers.data.data_collator import default_data_collator
from torch import nn
from typing import Dict, List
from datasets import load_dataset
import datasets
from datasets import Dataset


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/local/lib/python3.7/dist-packages/yaml/constructor.py:126: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 

In [ ]:
import os 
from google.colab import drive
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dataset1 = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumtrainDataset_1")
train_dataset2 = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumtrainDataset_2")
train_dataset3 = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumtrainDataset_3")
train_dataset4 = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumtrainDataset_4") ###train_dataset = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumtrainDataset_2")
valid_dataset1 = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumvalidDataset")
train_dataset5 = datasets.load_from_disk("/content/drive/MyDrive/orig/xsumtrainDataset") ###train_dataset = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumtrainDataset_2")
valid_dataset2 = datasets.load_from_disk("/content/drive/MyDrive/orig/xsumvalidDataset")
#teacher = torch.load('teacher_model.pt', map_location=torch.device('cuda'))#

In [ ]:
train1 = train_dataset1.to_pandas()
train2 = train_dataset2.to_pandas()
train3 = train_dataset3.to_pandas()
#train4 = train_dataset4.to_pandas()
train5 = train_dataset5.to_pandas()

In [ ]:
df_3 = pd.concat([train1, train2,train3[0:64000],train5[0:200000]])

In [ ]:
len(df_3['input_ids'])

400000

In [ ]:
from datasets import Dataset
train_data = Dataset.from_dict(train_dataset)

In [ ]:
from datasets import Dataset
train_data = Dataset.from_pandas(df_3)

In [ ]:
train_data = train_data.remove_columns('__index_level_0__')

In [ ]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask','decode_ids','decode_mask'], device='cuda')

In [ ]:
train_data.save_to_disk('/content/drive/MyDrive/full_data/pandas')